# Predicting house prices in Melbourne

In this small assignment for the lecture *Social Data Science*, we used the Melbournce house price dataset (https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market) to build a simple regression model, that predicts house prices.

First we will prepare the data, after that we are using the `statsmodels` library to perform least squares regression on different sets of columns and analyze our findings.

In [1]:
#We need the following libraries
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import statsmodels.api as sm
from typing import List
from statsmodels.regression.linear_model import RegressionResultsWrapper

## Load and prepare data 

The function `load_data` loads the Melbourne data and thereby

- discards extremely small ($\leq 10$) and large houses ($\geq 1500$) determined by the `'BuildingArea'`.
- drops the columns `['Address', "Lattitude", "Longtitude"]`. We are doing this because these features do not contain any further information (regarding the price) than `Suburb`.
- converts the `'Date'` column into a `'Days'` column, which measures the number of days from the start of first dataset.
- discard rows that contain non numeric (nan) values. 

In [2]:
def load_data(folder : Path) -> pd.DataFrame:
    file = "melb_data.csv"

    # read data and drop some instances
    df = pd.read_csv(folder/file)
    df.drop(df[(df['BuildingArea'] <= 10) | (df['BuildingArea'] >= 1500)].index, inplace = True)
    df.dropna(inplace = True)
    
    # convert date to days
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
    first_date = min(df["Date"])
    df["Days"] = (df["Date"]-first_date).apply(lambda x: getattr(x, "days"))
    
    # drop columns
    df = df.drop(['Address', "Lattitude", "Longtitude", "Date"], axis=1)
    return df

In [3]:
#load data (folder = data) and show some instances
df_melb = load_data(Path("data"))
df_melb.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount,Days
1,Abbotsford,2,h,1035000.0,S,Biggin,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,Northern Metropolitan,4019.0,0
2,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,Northern Metropolitan,4019.0,394
4,Abbotsford,4,h,1600000.0,VB,Nelson,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,Northern Metropolitan,4019.0,121
6,Abbotsford,3,h,1876000.0,S,Nelson,2.5,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,Yarra,Northern Metropolitan,4019.0,93
7,Abbotsford,2,h,1636000.0,S,Nelson,2.5,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,Yarra,Northern Metropolitan,4019.0,247


When doing regression it is usually bad practice if a column is a unique identifier. E.g. in the house pricing dataset there are some sellers that have sold only one house. Thus that datapoint can be uniquely explained with that particular seller.

To avoid this problem you the function `drop_almost_unique()` merges values of a column by replacing entries fewer than min_count with the entry `"other"`.

In [4]:
def drop_almost_unique(df: pd.DataFrame, column : str, min_count : int) -> pd.DataFrame:
    # df should be left unchanged! 
    df_dropped = df.copy()
    value_counts = df[column].value_counts().items()
    for seller, count in value_counts:
        if(count < min_count):
            df_dropped[column].replace(seller, 'other', inplace = True)
    return df_dropped

We later want to perform least squares regression on different sets of descriptive features:

- Numeric: `['Rooms', 'Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Days']`
- Sales: `['Type', 'Method', 'Seller']`
- Suburbs: `['Suburb']`
- All: Numeric + Sales + Suburbs

To prepare the DataFrame for fitting, the function `prepare_df` filters the columns of the dataframe to `cols` and the Price column. It further uses `drop_almost_unique` on the `SellerG` and `Suburb` column with the provided `min_count` to make sure that no unique identifiers get through. To prepare the non numeric attributes we use `pd.get_dummies` and drop specific columns  indicated in the dictionary (`dummy_drop_dict`) below. Preparation should leave the original input dataframe untouched.

In [5]:
#define sets of columns
numeric_cols = ['Rooms', 'Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Days']
sales_cols = ["Type", "Method", "SellerG"]
suburb_cols = ["Suburb"]
all_cols = numeric_cols + sales_cols + suburb_cols

In [6]:
#specify which columns to drop
dummy_drop_dict = {
    "SellerG" : "other",
    "Suburb" : "other",
    "Type" : "h",
    "Method" : "S"
}

In [7]:
def prepare_df(df_in : pd.DataFrame, cols : List[str], min_count : int) -> pd.DataFrame:
    df_out = df_in[cols + ["Price"]].copy()
    if "SellerG" in cols:
        df_out = drop_almost_unique(df_out, "SellerG", min_count)
    if "Suburb" in cols:
        df_out = drop_almost_unique(df_out, "Suburb", min_count)
    for column in dummy_drop_dict:
        if column in cols:
            df_out = pd.get_dummies(df_out, columns = [column]).drop(column + "_" + dummy_drop_dict[column], axis=1)
    return df_out

In [8]:
#example output of prepare_df
prepare_df(df_melb, numeric_cols, min_count=100).head()

,Rooms,Distance,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Days,Price
1,2,2.5,1.0,0.0,156.0,79.0,1900.0,0,1035000.0
2,3,2.5,2.0,0.0,134.0,150.0,1900.0,394,1465000.0
4,4,2.5,1.0,2.0,120.0,142.0,2014.0,121,1600000.0
6,3,2.5,2.0,0.0,245.0,210.0,1910.0,93,1876000.0
7,2,2.5,1.0,2.0,256.0,107.0,1890.0,247,1636000.0


## Least squares fitting 

We now use the `statsmodels` library to perform least squares regression on the different sets of columns.

The function `perform_fits()` fits all four of these models to the provided dataset. The function returns the four trained models and a list of mean absolute error (MAE, $\sum_i |y_i - \hat f(x_i)|$) for the particular model.




In [9]:
def perform_fits(df_in : pd.DataFrame, min_count : int) -> (List[RegressionResultsWrapper], List[float]):
    # define different data sets
    df_numeric = sm.add_constant(prepare_df(df_in, numeric_cols, min_count))
    df_sales = sm.add_constant(prepare_df(df_in, sales_cols, min_count))
    df_suburb = sm.add_constant(prepare_df(df_in, suburb_cols, min_count))
    df_all = sm.add_constant(prepare_df(df_in, all_cols, min_count))
    
    # create models
    models = [sm.OLS(df_numeric.Price, df_numeric.drop("Price", axis = 1)).fit(),
             sm.OLS(df_sales.Price, df_sales.drop("Price", axis = 1)).fit(),
             sm.OLS(df_suburb.Price, df_suburb.drop("Price", axis = 1)).fit(),
             sm.OLS(df_all.Price, df_all.drop("Price", axis = 1)).fit()]
    
    # compute MAE
    diffs = [models[0].predict(df_numeric.drop("Price", axis = 1))-df_numeric.Price,
            models[1].predict(df_sales.drop("Price", axis = 1))-df_sales.Price,
            models[2].predict(df_suburb.drop("Price", axis = 1))-df_suburb.Price,
            models[3].predict(df_all.drop("Price", axis = 1))-df_all.Price]
    MAEs = [np.mean(np.abs(x)) for x in diffs]
    return models, MAEs

In [10]:
# fit models and compute mean absolute errors for the respective models
(Numeric, Sales, Suburbs, All), MAEs = perform_fits(df_melb, 2)
MAEs

[287421.6719700343, 343449.5484080682, 362614.8630186428, 212256.37106902784]

In [11]:
Numeric.summary() #change for different models

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.561
Method:                 Least Squares   F-statistic:                     983.5
Date:                Sun, 04 Dec 2022   Prob (F-statistic):               0.00
Time:                        19:08:32   Log-Likelihood:                -88874.
No. Observations:                6159   AIC:                         1.778e+05
Df Residuals:                    6150   BIC:                         1.778e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1.022e+07    3.2e+05     31.882      0.000    9.59e+06    1.08e+07
Rooms         1.099e+05   9234.972     11.903      0.000    9.18e+04    1.28e+05
Distance     -3.013e+04   1164.998    -25.859      0.000   -3.24e+04   -2.78e+04
Bathroom      2.236e+05   1.12e+04     19.970      0.000    2.02e+05    2.46e+05
Car           5.563e+04   6992.604      7.955      0.000    4.19e+04    6.93e+04
Landsize        18.2309      6.416      2.842      0.005       5.654      30.808
BuildingArea  3026.9120    103.253     29.316      0.000    2824.501    3229.323
YearBuilt    -5145.4545    163.701    -31.432      0.000   -5466.365   -4824.544
Days           165.1630     39.318      4.201      0.000      88.085     242.241
==============================================================================
Omnibus:                     4123.302   Durbin-Watson:                   1.518
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           269396.478
Skew:                           2.502   Prob(JB):                         0.00
Kurtosis:                      35.011   Cond. No.                     1.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Results

The model All performs best in terms of R^2 (0.757) as well as MAE (212256.371), which is not surprising since the other models use subsets of this models descriptive features. Assuming the prices are 
given in Dollar this means that on average the model predicts a price that differs from the actual price by more than 
200.000 $. We think this is quit much. To be more usefull for predicting house prices, we might consider more variables or maybe house prices depend too much
on circumstances that cannot be measured.

To learn more about the house market in Melbourne, we think the suburbs model is actually the most interesting.
Considering that this model only uses one column, it performs surprisingly well (although it is the worst of the four models.)
This shows that the house prices heavily depend on the location.
The parameters of the model are very easy to interpret and tell us in which areas houses are more expensive.
Of course it is important to note that we cannot say if the location itself has influence on the price. It could be that houses
in certain areas just tend to be bigger for example. 
The numeric model also performs pretty well, but does not tell us that much about Melbourne specifically. The information we
get from the parameters (such as more rooms correlate with higher prices) are also not very surprising.